In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("datos_perfumes_douglas.csv")

In [3]:
df = df.drop('Unnamed: 0', axis=1)

In [4]:
df.insert(0, 'tienda', "douglas")

In [5]:
def convertir_a_minusculas(valor):
    if isinstance(valor, str):
        return valor.lower()
    return valor

df = df.map(convertir_a_minusculas)

In [6]:
df['categoria'] = df['categoria'].str.replace('fragancias de hombre', 'hombre')
df['categoria'] = df['categoria'].str.replace('fragancias de mujer', 'mujer')

In [7]:
def mapear_tipos_y_limpiar(df):    

    mapping = {
        'perfume'                : 'parfum',
        'eau de  parfum'         : 'eau de parfum',
        'eau de  parfum'         : 'eau de parfum',
        'eau de  parfum'         : 'eau de parfum',
        'eau de  parfum'         : 'eau de parfum',
        'eau de  parfum'         : 'eau de parfum',
        'eau de  parfum'         : 'eau de parfum',
        'eau de  parfum'         : 'eau de parfum',
        'eau parfumee'           : 'eau de parfum',
        'eau parfumeé'           : 'eau de parfum',
        'extrait de parfum'      : 'parfum',
        'essence de parfum'      : 'parfum',
        'eau detoilette'         : 'eau de toilette',
        'eau fraîche'            : 'eau de cologne',
        'para hombre'            : '',
        'para mujer'             : '',
        'para  mujer'            : '',
        'unisex'                 : '',
        'intense'                : '',
        'extreme'                : '',
        'légère'                 : '',
        'cologne spray'          : '',
        'el ritual perfumado -'  : '',
        'y vaporizador de viaje' : ''
    }

    for name, replacement in mapping.items():
        df['tipo'] = df['tipo'].str.replace(name, replacement, regex=False)

    df['tipo'] = df['tipo'].str.strip()
    
    return df

mapear_tipos_y_limpiar(df)

,tienda,categoria,marca,nombre,tipo,precio_sin_desc,descuento,precio_con_desc,valoracion,num_valoraciones
0,douglas,hombre,chanel,bleu de chanel,eau de parfum,"127,99 €",no aparece,sin descuento,5.0,1.0
1,douglas,hombre,chanel,allure homme,eau de toilette,"115,99 €",no aparece,sin descuento,NaN,NaN
2,douglas,hombre,yves saint laurent,la nuit de l’homme,eau de toilette,"54,95 €",no aparece,"38,99 €",5.0,2.0
3,douglas,hombre,armani,acqua di giò,eau de toilette,"70,99 €",no aparece,"40,99 €",5.0,2290.0
4,douglas,hombre,hugo boss,boss bottled,eau de toilette,"71,99 €",no aparece,"29,99 €",5.0,3.0
...,...,...,...,...,...,...,...,...,...,...
3695,douglas,mujer,jil sander,sun,gel de ducha,"9,99 €",no aparece,sin descuento,NaN,NaN
3696,douglas,mujer,van cleef & arpels,first,eau de parfum,"124,99 €",no aparece,sin descuento,NaN,NaN
3697,douglas,mujer,dior,joy by dior,desodorante,"49,99 €",no aparece,sin descuento,NaN,NaN
3698,douglas,mujer,chanel,coco,eau de parfum,"123,99 €",no aparece,sin descuento,NaN,NaN


In [8]:
df.loc[df['tipo'] == 'cologne', 'tipo'] = 'eau de cologne'

In [9]:
def clean_prices(df, columns):
    
    for col in columns:
        
        df[col] = df[col].str.replace('[^0-9,]', '', regex=True) 
        df[col] = df[col].str.replace(',', '.', regex=True)
        df[col] = df[col].replace('', '0', regex=True)
        df[col] = df[col].astype(float)  

columns = ['descuento', 'precio_con_desc','precio_sin_desc']

clean_prices(df, columns)

In [10]:
columns = ['valoracion', 'num_valoraciones']

for col in columns:
    df[col] = df[col].replace(['sin valoracion', 'sin valoraciones'], np.nan)

In [11]:
df.to_csv('datos_perfumes_douglas_filtrados.csv', index=False)

In [12]:
df['tipo'].value_counts()

tipo
eau de parfum                     1671
eau de toilette                   1167
set de fragancia                   219
parfum                             187
eau de cologne                     145
aftershave                          78
loción corporal                     51
spray corporal                      46
desodorante                         32
gel de ducha                        21
parfum para cabello                 16
gel de ducha perfumado              14
cuidado anticelulítico               9
calendario de adviento               5
crema para el cuerpo                 4
vaporizador                          4
eau fraiche                          3
cofre higiene personal               3
champú capilar                       2
tratamiento corporal perfumado       2
jabón                                2
crema facial                         2
cofre                                1
fluido corporal                      1
maquinilla de afeitar                1
toalla              

In [13]:
# Función para actualizar Con Descuento
def actualizar_valor(row):
    if row['precio_con_desc'] == 0.00:
        return row['precio_sin_desc']
    else:
        return row['precio_con_desc']

# Aplicar la función a cada fila del DataFrame
df['precio_con_desc'] = df.apply(actualizar_valor, axis=1)

In [14]:
df['descuento'] = ((df['precio_sin_desc'] - df['precio_con_desc']) / df['precio_sin_desc']) * 100
df['descuento'] = df['descuento'].round(2)

In [15]:
df = df.drop_duplicates()

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2904 entries, 0 to 3697
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tienda            2904 non-null   object 
 1   categoria         2904 non-null   object 
 2   marca             2769 non-null   object 
 3   nombre            2904 non-null   object 
 4   tipo              2898 non-null   object 
 5   precio_sin_desc   2904 non-null   float64
 6   descuento         2904 non-null   float64
 7   precio_con_desc   2904 non-null   float64
 8   valoracion        421 non-null    float64
 9   num_valoraciones  421 non-null    float64
dtypes: float64(5), object(5)
memory usage: 249.6+ KB


In [17]:
lista_tipos = ['eau de parfum', 'eau de toilette', 'parfum', 'eau de cologne']
df = df[df["tipo"].isin(lista_tipos)].reset_index(drop=True)

In [18]:
df

,tienda,categoria,marca,nombre,tipo,precio_sin_desc,descuento,precio_con_desc,valoracion,num_valoraciones
0,douglas,hombre,chanel,bleu de chanel,eau de parfum,127.99,0.00,127.99,5.0,1.0
1,douglas,hombre,chanel,allure homme,eau de toilette,115.99,0.00,115.99,NaN,NaN
2,douglas,hombre,yves saint laurent,la nuit de l’homme,eau de toilette,54.95,29.04,38.99,5.0,2.0
3,douglas,hombre,armani,acqua di giò,eau de toilette,70.99,42.26,40.99,5.0,2290.0
4,douglas,hombre,hugo boss,boss bottled,eau de toilette,71.99,58.34,29.99,5.0,3.0
...,...,...,...,...,...,...,...,...,...,...
2417,douglas,mujer,yves saint laurent,ysl klassiker,eau de parfum,160.00,61.25,62.00,NaN,NaN
2418,douglas,mujer,yves saint laurent,ysl klassiker,eau de parfum,53.00,0.00,53.00,NaN,NaN
2419,douglas,mujer,van cleef & arpels,first,eau de toilette,116.99,0.00,116.99,NaN,NaN
2420,douglas,mujer,dior,j’adore,eau de parfum,59.99,0.00,59.99,5.0,1.0


In [19]:
df.to_csv('datos_perfumes_douglas_filtrados.csv', index=False)